## Mock

mock 库中最常用的类型和函数应属 ``mock.Mock`` 和 ``patch``，其它，比如 ``MagicMock`` 也有比较重要的作用。


In [2]:
from mock import Mock, MagicMock, patch

In [3]:
class ProductionClass(object):
    def some_method(self):
        # doing something~~
        self.do_something(1, 2, 3)
    def do_something(self, a, b, c):
        # balabalabala
        pass

## 基本使用

现在我们使用 Mock 对象替换 ``ProductionClass`` 的 ``do_something`` 方法：

In [6]:
ProductionClass.do_something = Mock()
p = ProductionClass()
p.some_method()

In [7]:
p.do_something.assert_called_once_with(1, 2, 3)
p.do_something.assert_called_once_with(1, 2, 2) # 这里会报错，因为没有传入过这样的参数

AssertionError: Expected call: mock(1, 2, 2)
Actual call: mock(1, 2, 3)

## Mock 的使用

### Mock 的参数和属性

* return_value 生成的 ``Mock`` 实例永远返回该值
* side_effect  （值为函数时）使用 ``side_effect`` 函数代替被 mock 的方法，如存在则覆盖 ``return_value``
* side_effect  （值为 iterable 时）每次执行被 mock 的方法都返回 ``side_effect`` 中下一个值
* spec         mock 对象的模板，``spec`` 不存在的属性不会被 mock，调用时会报错

In [8]:
p.do_something.return_value = 2
print p.do_something()

2


In [9]:
print p.do_something()
p.do_something.side_effect = lambda : 8
print p.do_something()

2
8


In [13]:
from random import  randint
ProductionClass.do_something = Mock(return_value=2, side_effect=lambda : randint(0, 8))
p2 = ProductionClass()
print p2.do_something()

1


In [14]:
MockedClass = Mock()
print MockedClass.invalid_method()
MockedClass = Mock(spec=ProductionClass)
print MockedClass.invalid_method()

<Mock name='mock.invalid_method()' id='4413769744'>


AttributeError: Mock object has no attribute 'invalid_method'

### Mock 的测试方法

* assert_called_with 断言最后一次调用时的参数是否一致
* assert_called_once_with 断言该对象仅被调用一次，并且参数为 ``*args`` ``**kwargs``
* assert_any_call 只需要被 mock 对象有一次调用参数与之相同

In [15]:
class NewClass(object):
    pass
    
def do_something(self, *args, **kw):
    pass
NewClass.do_something = Mock(side_effect=do_something)

nc = NewClass()
nc.do_something(1, 2, 3)

nc.do_something.assert_called_once_with(1, 2, 3)
nc.do_something.assert_called_with(1, 2, 3)

In [16]:
nc.do_something(1, 2, 3)

nc.do_something.assert_called_with(1, 2, 3)
print u'do_something 已被调用 %d 次' % nc.do_something.call_count
nc.do_something.assert_called_once_with(1, 2, 3)

do_something 已被调用 4 次


AssertionError: Expected to be called once. Called 4 times.

## Mock 与 MagicMock

Mock 与 MagicMock 的不同之处在于，Mock 不会对被 mock 对象的魔术方法（Magic Method）进行任何处理。

In [17]:
mocked_obj = Mock()
mocked_obj[3] = 'whatever'

TypeError: 'Mock' object does not support item assignment

In [18]:
mocked_obj = MagicMock()
mocked_obj[3] = 'whatever'
mocked_obj.__setitem__.assert_called_with(3, 'whatever')
print mocked_obj[3]

<MagicMock name='mock.__getitem__()' id='4414761744'>

## ``patch`` 装饰器

patch 是个装饰器，是个 callable object，提供了一种更优雅的 mock 手段。

比如，在测试 model 时可以这样：

```python
from unittest import TestCase, main
from app.models import User

class FakeTest(TestCase):
    @patch('__main__.User', FakeClass)
    def test_whatever(self):
        user = User()
```

In [20]:
def fake_time():
    return 111111111
class FakeClass(object):
    pass

import time
from UserDict import UserDict
@patch('time.time', fake_time)
@patch('UserDict.UserDict', FakeClass)
def test():
    print UserDict
    return time.time()

test()

UserDict.UserDict


111111111